# "Answer Business Questions with Data Science - SQL case study"
> "Using SQLite"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [SQL, business]
- image: images/SQL.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

# 1. Introduction

## 1.1. Problem Definition

One of the most often ignored part of any data science project is defining which problem your code implementation is trying to solve. In engineering college I was trained to solve problems which were already framed for me – my task was just using a set of constrained manipulations to find the variable that was missing.  

In real life, problems does not appear in such organized manner. They are generally a fuzzy collection of useful and useless information on which we need to reflect upon and include what is important and exclude what is not.

Here, I will define the term **problem** as an offset between a *present state* and a *desired state*. What we do to go from a current to a desired state is the problem solution! The technology employed to solve the problem may vary depending on how you frame it and which solution path you choose. 

Next I will provide the context for the problem we will solve and derive a simple statement from it (simple subject + action verb + object). Than we can go from a broad description to a statement with a call for action:


**Context:** Suppose you are working on a digital media record store that sells tracks and entire albuns of several artists and music genres across several countries of the world. Supposing the record stores has made a deal with a record label. They claim that they have **FOUR INCREDIBLE ALBUNS** that will make a huge success in the **USA** and want to advertise them. However, we can only include three of them in the Store. These albuns are from four distinct genres which are already found in several tracks in our digital store - **'Hip Hop/Rap','Alternative & Punk','Pop','Blues'**

**Problem statement:** Select 3 out of 4 possible albuns to include in the digital store.

## 1.2. Solution Strategy

As you may noticed, there are several ways I can select 3 out of 4 albuns – each will recquire a certain amount of resources (time, technology). Let's assess them in this terms:

1.  Write the genres on pieces of paper, put them in a box, and draw 3 of them randomly - **Resources: low, Results: Probably poor**. 

2.  Try assigning a meaningful metric to each possible triplet of albuns and maximize/minimize it. **Resources: medium, Results: ok**.

Here I will choose the 2nd option since Data Science is about answering questions based on data and not lucky! Hopefully, this approach is better than pure chance. 

### The Data Base

The data base used is this analysis is the [Chinook](https://github.com/lerocha/chinook-database/blob/master/README.md) data model which represents a digital media store, including tables for artists, albums, media tracks, invoices and customers. A copy of the database schema is displayed below.

![chinook](https://raw.githubusercontent.com/viniviena/repositoryname/master/Chinook.png)

To implement the 2nd option we need a metric to assign for each triplet. In the database, we have few information we can use. Basically we want our choice to maximize the chance that the selected albuns will be sold as much as the ones we already have in our store. There are two words to pay attention: **"chance"** and **"will be sold"**. 

Chances triggers my mind to probability and counting - maybe using genre popularity is a way. About the the verb form "will be sold". It is too vague - when we talk about the future it is desirable to set a time frame (1 one month, pi years, 3.74 years).

As you may noticed, the database schema does not provide any time-related information about the invoices, albuns or artists. This could be valuable for deciding if something will be succesful in a near future. Even so, we can still use available information to guide our decision.  

If you notice, only one attribute of this new album was provided - their genre. We know that music has much more attributes than its genre - I know several artists that play songs from the same genre but sounds quite different. Even so, lets use what we have.

One approach to solve this problem is collecting information about the **popularity** or **total sales** of those genres in our store and pick the triplet which has the highest popularity.

This may seem a little biased since popularity vary over time and by lumping everything we can draw bad conclusions. However, this is the information we have at hand. **Here I'll use the total sales since it is correlated with popularity but it is easier to interpret and it is what guide any company (profit)** 


**Finally, we will choose 3 albuns for which the sum of total sales is maximized.**

# 2. Solution implementation

## 2.1. Importing modules

In [1]:
#collapse-hide
%matplotlib inline
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

## 2.2. Creating support functions

1. `run_query()` - takes a SQL query as argument and returns a panda dataframe contaning the correspondent data.

2. `run_command()` - takes a SQL command and execute using sqlite module.

3. `show_tables()` - Function that calls run_query() function to return a list of all tables and views in the database.

In [2]:
#collapse-hide
# Function 1 runquery
def run_query(q):
    with sql.connect('chinook.sqlite') as conn:
        return pd.read_sql(q,conn)
    
#Function 2 run_command    
def run_command(c):
    with sql.connect('chinook.sqlite') as conn:
        conn.isolation_level = None
        conn.execute(c)
        
#Function 3 show_tables

def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

In [3]:
show_tables()

,name,type
0,Album,table
1,Artist,table
2,Customer,table
3,Employee,table
4,Genre,table
5,Invoice,table
6,InvoiceLine,table
7,MediaType,table
8,Playlist,table
9,PlaylistTrack,table


In order to answer which three albuns include in the Store, we can create three queries:

* The **first subquery** join the *invoice, invoice_inline and customer* tables considering only customers from USA and return the `total` and `track_id` columns. 


* The **second subquery** join the *track* table with the *genre* table considering only the genres HIP-HOP,PUNK,POP and BLUES. it return `track_id` and `genre_name` columns. 


* The final query join the two queries and apply the `SUM()` operator in the total_USA column from subquery 1 and group by genre_name.

In [4]:
#collapse-hide
#Testing subqueries
#SQ1
sq1 = '''SELECT
            invl.TrackId track_id,
            inv.total total_USA
            FROM customer c
            LEFT JOIN invoice inv ON c.CustomerId = inv.CustomerId
            LEFT JOIN InvoiceLine invl ON invl.InvoiceId = inv.invoiceId
            WHERE c.country = "USA"
'''
print('The first ten rows of subquery 1 are')
table_q1 = run_query(sq1)
run_query(sq1).head(10)

The first ten rows of subquery 1 are


,track_id,total_USA
0,462,0.99
1,907,1.98
2,909,1.98
3,1236,13.86
4,1245,13.86
5,1254,13.86
6,1263,13.86
7,1272,13.86
8,1281,13.86
9,1290,13.86


In [5]:
#calculating the sum of total invoices in the USA for all genres
sum_total = table_q1['total_USA'].sum()

In [6]:
#collapse-hide
#Testing Subqueries
#SQ2
sq2= '''    SELECT 
               t.TrackId,
               g.name genre_name
               FROM genre g
               LEFT JOIN track t ON t.GenreId = g.GenreId
               WHERE genre_name IN ('Hip Hop/Rap','Alternative & Punk','Pop','Blues')

'''
print('the first 10 rows of subquery 2 are:')
run_query(sq2).head(10)

the first 10 rows of subquery 2 are:


,TrackId,genre_name
0,99,Alternative & Punk
1,100,Alternative & Punk
2,101,Alternative & Punk
3,102,Alternative & Punk
4,103,Alternative & Punk
5,104,Alternative & Punk
6,105,Alternative & Punk
7,106,Alternative & Punk
8,107,Alternative & Punk
9,108,Alternative & Punk


In [7]:
#collapse-hide
#Final results
q1_sq1 = '''
        WITH sq1 AS 
        
            (
            SELECT
            invl.TrackId track_id,
            inv.Total total_USA
            FROM customer c
            LEFT JOIN invoice inv ON c.CustomerId = inv.CustomerId
            LEFT JOIN InvoiceLine invl ON invl.InvoiceId = inv.InvoiceId
            WHERE c.country = "USA"
                    ),
                    
            sq2 AS 
            
                (
               SELECT 
               t.TrackId,
               g.name genre_name
               FROM genre g
               LEFT JOIN track t ON t.GenreId = g.GenreId
               WHERE genre_name IN ('Hip Hop/Rap','Alternative & Punk','Pop','Blues')
                )
                
            SELECT 
            sq1.track_id,
            sq2.genre_name,
            SUM(sq1.total_USA) total
            FROM sq1
            INNER JOIN sq2 ON sq2.TrackId = sq1.track_id
            GROUP BY genre_name
            ORDER BY total DESC
 '''
final_results = run_query(q1_sq1)

print('the final query table is:')
final_results

the final query table is:


,track_id,genre_name,total
0,99,Alternative & Punk,415.92
1,198,Blues,126.72
2,3467,Pop,33.66
3,2249,Hip Hop/Rap,6.93


It is useful to include a column with the total purchased for each genre in percetage of the total purchased in the USA for all genres. Thus, a new column is introduced by applying the following equation:


$total(percent) = \frac{total}{sumtotal}$

In [8]:
final_results_perc = final_results.copy()
sum_total = sum(final_results_perc['total'].values)
final_results_perc['total_%'] = final_results_perc['total']/sum_total*100
#final_results_perc.set_index('genre_name',inplace=True)
final_results_perc

,track_id,genre_name,total,total_%
0,99,Alternative & Punk,415.92,71.313204
1,198,Blues,126.72,21.727277
2,3467,Pop,33.66,5.771308
3,2249,Hip Hop/Rap,6.93,1.188210


## 3. Answering our question.
As you can see, the 3 genres that maximizes the total sales are **Alternative & Punk, Blues and Pop**. Remember that I have assumed that the total sales maximization is a good indicator of future success of new albuns with these attributes.

* The question can be answered using the final query table. However, we can say that this result could be part of a report in which the graphical display of information is very desirable. In this way, we can plot a bar chart displaying the results arised from question 1 as following:

In [9]:
#Importing plotly modules
from IPython.display import HTML
import plotly_express as px


In [10]:
#hide
#Chart with the results of the previous table
def plot():
    graph = px.bar(final_results_perc,
                  y='genre_name',
                  x='total_%',
                  text=final_results_perc['total_%'].round(2),
                  orientation = 'h',
                  width=500, 
                   height=300
                   )
    graph.update_layout(title = 'Answering question 1',
                       title_font_size = 20,
                      plot_bgcolor = 'rgba(0,0,0,0)',
                       xaxis = dict(
                                   showgrid=False,
                                   showline=False,
                                   showticklabels=True,
                                   zeroline=False,
                                   title_text = None),

                       yaxis = dict(
                                   showgrid=False,
                                   showline=False,
                                   showticklabels=True,
                                   zeroline=False,
                                   title_text = None),

                      )

    graph.update_traces(textposition = 'outside',
                      marker = dict(
                                    line = dict(color = 'rgba(58, 71, 80, 1)',width = 2
                                               )
                                       )
                      )
    return graph


In [11]:
figure = plot()
HTML(figure.to_html(include_plotlyjs='cdn'))

## Conclusions

Let's recap what we did in this post:

1) We framed the problem to be solved and mapped two possible solutions. 

2) Then, we pick up one that can give us an acceptable answer. 

3) Then, we prepared few queries to gather information about the total sales of tracks sold in the USA within the 4 possible genres.

4) Finally, we ranked the genres total sales in descending order and selected the top 3.